# Yield-Driven Capacity Expansion Model
## 11 GW → 18 GW Manufacturing Amplification

This notebook models the multiplicative yield cascade across:
- Puller yield & cycle time
- Wafer thickness & kerf reduction
- Ingot-to-brick loss
- Wafering yield

Objective:
Quantify system-level capacity amplification without increasing puller count.


In [1]:
import numpy as np
import pandas as pd

# ---------------------------
# Fixed Parameters
# ---------------------------

pullers = 672
uptime = 0.90
ingots_per_cycle = 5
hours_per_year = 8760

# ---------------------------
# Baseline Parameters
# ---------------------------

yield_puller_1 = 0.902
cycle_time_1 = 318  # hours
usable_length_1 = 2833  # mm
thickness_1 = 0.130  # mm
kerf_1 = 0.055  # mm
brick_yield_1 = 1 - 0.081
wafer_yield_1 = 0.925

# ---------------------------
# Improved Parameters
# ---------------------------

yield_puller_2 = 0.926
cycle_time_2 = 281  # hours
usable_length_2 = 2908  # mm
thickness_2 = 0.110  # mm
kerf_2 = 0.045  # mm
brick_yield_2 = 1 - 0.044
wafer_yield_2 = 0.95


In [2]:
# ---------------------------
# Function: Ingots per puller per year
# ---------------------------

def ingots_per_puller(yield_puller, cycle_time):
    effective_hours = hours_per_year * uptime
    cycles_per_year = effective_hours / cycle_time
    ingots_year = cycles_per_year * ingots_per_cycle * yield_puller
    return ingots_year

# Compute
ingots_1 = ingots_per_puller(yield_puller_1, cycle_time_1)
ingots_2 = ingots_per_puller(yield_puller_2, cycle_time_2)

print("Baseline ingots per puller:", round(ingots_1, 2))
print("Improved ingots per puller:", round(ingots_2, 2))
print("Puller amplification ratio:", round(ingots_2 / ingots_1, 3))


Baseline ingots per puller: 111.81
Improved ingots per puller: 129.9
Puller amplification ratio: 1.162


In [3]:
# ---------------------------
# Function: Wafers per ingot
# ---------------------------

def wafers_per_ingot(usable_length, thickness, kerf, brick_yield, wafer_yield):
    raw_wafers = usable_length / (thickness + kerf)
    after_brick = raw_wafers * brick_yield
    usable_wafers = after_brick * wafer_yield
    return usable_wafers

# Compute
wafers_1 = wafers_per_ingot(
    usable_length_1, thickness_1, kerf_1,
    brick_yield_1, wafer_yield_1
)

wafers_2 = wafers_per_ingot(
    usable_length_2, thickness_2, kerf_2,
    brick_yield_2, wafer_yield_2
)

print("Baseline wafers per ingot:", round(wafers_1, 0))
print("Improved wafers per ingot:", round(wafers_2, 0))
print("Wafer amplification ratio:", round(wafers_2 / wafers_1, 3))


Baseline wafers per ingot: 13018.0
Improved wafers per ingot: 17039.0
Wafer amplification ratio: 1.309


In [4]:
# ---------------------------
# Total ingots per year
# ---------------------------

total_ingots_1 = ingots_1 * pullers
total_ingots_2 = ingots_2 * pullers

# ---------------------------
# Total wafers per year
# ---------------------------

total_wafers_1 = total_ingots_1 * wafers_1
total_wafers_2 = total_ingots_2 * wafers_2

print("Baseline wafers/year:", format(int(total_wafers_1), ","))
print("Improved wafers/year:", format(int(total_wafers_2), ","))

print("\nTotal system amplification ratio:",
      round(total_wafers_2 / total_wafers_1, 3))


Baseline wafers/year: 978,131,850
Improved wafers/year: 1,487,423,920

Total system amplification ratio: 1.521


In [5]:
# ---------------------------
# Convert Full Wafers to GW
# ---------------------------

power_per_half_wafer = 5.09  # W

# Convert full wafers → half wafers
half_wafers_1 = total_wafers_1 * 2
half_wafers_2 = total_wafers_2 * 2

# Total power in watts
total_power_1 = half_wafers_1 * power_per_half_wafer
total_power_2 = half_wafers_2 * power_per_half_wafer

# Convert to GW
gw_1 = total_power_1 / 1e9
gw_2 = total_power_2 / 1e9

print("Baseline capacity (GW):", round(gw_1, 2))
print("Improved capacity (GW):", round(gw_2, 2))
print("Capacity amplification ratio:", round(gw_2 / gw_1, 3))
print("Absolute GW gain:", round(gw_2 - gw_1, 2))


Baseline capacity (GW): 9.96
Improved capacity (GW): 15.14
Capacity amplification ratio: 1.521
Absolute GW gain: 5.18


In [6]:
# ---------------------------
# Pullers required without improvements
# ---------------------------

required_pullers_baseline = (gw_2 / gw_1) * pullers

print("Pullers required at baseline efficiency:",
      round(required_pullers_baseline))

print("Current pullers:", pullers)

pullers_avoided = required_pullers_baseline - pullers

print("Pullers avoided due to yield improvements:",
      round(pullers_avoided))


Pullers required at baseline efficiency: 1022
Current pullers: 672
Pullers avoided due to yield improvements: 350


In [8]:
# ---------------------------
# Capex Avoidance Calculation
# ---------------------------

capex_per_puller = 300000  # USD

capex_avoided = pullers_avoided * capex_per_puller

print("Pullers avoided:", round(pullers_avoided))
print("Capex avoided (USD):", format(int(capex_avoided), ","))
print("Capex avoided (Million USD):",
      round(capex_avoided / 1e6, 2))


Pullers avoided: 350
Capex avoided (USD): 104,968,753
Capex avoided (Million USD): 104.97


In [9]:
# ---------------------------
# Energy Avoidance Calculation
# ---------------------------

# Energy per ingot (kWh)
energy_per_ingot = (200 * 18) + (70 * 48)

# Baseline ingots per puller
ingots_baseline = ingots_1

# Total ingots avoided per year
ingots_avoided_per_year = pullers_avoided * ingots_baseline

# Total energy avoided per year
energy_avoided_kwh = ingots_avoided_per_year * energy_per_ingot

print("Energy per ingot (kWh):", energy_per_ingot)
print("Annual ingots avoided:", round(ingots_avoided_per_year))
print("Annual energy avoided (MWh):",
      round(energy_avoided_kwh / 1000))
print("Annual energy avoided (GWh):",
      round(energy_avoided_kwh / 1e6, 2))


Energy per ingot (kWh): 6960
Annual ingots avoided: 39123
Annual energy avoided (MWh): 272298
Annual energy avoided (GWh): 272.3


In [10]:
# ---------------------------
# Annual Electricity Cost Avoided
# ---------------------------

cost_per_kwh_inr = 7  # INR per kWh

annual_cost_avoided_inr = energy_avoided_kwh * cost_per_kwh_inr

print("Annual electricity cost avoided (INR):",
      format(int(annual_cost_avoided_inr), ","))

print("Annual electricity cost avoided (Crore INR):",
      round(annual_cost_avoided_inr / 1e7, 2))


Annual electricity cost avoided (INR): 1,906,084,295
Annual electricity cost avoided (Crore INR): 190.61


## Executive Impact Summary

Process improvements (puller yield + cycle time + wafer thickness + kerf + yield cascade) resulted in:

• Pullers avoided: ~350 units  
• Direct Capex avoided: ~$105M  
• Annual energy avoided: ~272 GWh  
• Annual electricity cost avoided: ~₹190 Crore  

Strategic Insight:
Yield improvement and wafer process optimization provide capital-equivalent expansion 
without physical infrastructure growth.

This demonstrates how upstream engineering improvements cascade into 
large-scale economic and operational impact.
